In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import userdata
token = userdata.get('GITHUB_TOKEN')
user_name = userdata.get('GITHUB_USERNAME')
mail = userdata.get('GITHUB_MAIL')

!git config --global user.name "{user_name}"
!git config --global user.email "{mail}"
!git clone https://{token}@github.com/azhgh22/Walmart-Recruiting-Store-Sales-Forecasting.git

%cd Walmart-Recruiting-Store-Sales-Forecasting

from google.colab import userdata
! pip install -r ./requirements.txt
kaggle_json_path = userdata.get('KAGGLE_JSON_PATH')
! ./src/data_loader.sh -f {kaggle_json_path}

In [1]:
# **Torch**

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.config import *

stores = pd.read_csv(STORES_PATH)
features = pd.read_csv(FEATURES_PATH)
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

train['Date'] = pd.to_datetime(train.Date)

from src.processing import split_data
x_train,y_train,x_val,y_val = split_data(train)


Matplotlib is building the font cache; this may take a moment.


ModuleNotFoundError: No module named 'seaborn'

In [ ]:
## **Data creation**

def create_data_nbeats(x_train:pd.DataFrame,y_train:pd.DataFrame) -> pd.DataFrame:
    df = pd.DataFrame()

    df = df.reset_index()
    df["Date"] = x_train.Date
    df["Store"] = x_train.Store
    df["Dept"] = x_train.Dept
    df["y"] = y_train.values

    df.rename(columns={"Date": "ds"}, inplace=True)
    df["unique_id"] = df["Store"].astype(str) + "_" + df["Dept"].astype(str)

    df = df[["unique_id", "ds", "y"]].copy()
    df = df.sort_values(by=["ds"])
    # df.reset_index(drop=True, inplace=True)
    return df


In [ ]:
import torch
from neuralforecast.models import NBEATS
from neuralforecast.losses.pytorch import MSE
import logging

df = create_data_nbeats(x_train,y_train)

model = NBEATS(
    h=48,
    input_size=52,
    loss=MSE(),
    max_steps=10*104,
    batch_size=32,
    learning_rate=0.001,
    shared_weights=True,
    stack_types=('identity','trend','identity','seasonality', 'identity','identity'),
    n_blocks = [3,3,3,3,3,3],
    random_seed=42,
    accelerator='gpu',     # <--- Correct: This is an NBEATS argument now
    devices=1,             # <--- Correct: This is an NBEATS argument now
    logger=False,          # <--- Correct: This is an NBEATS argument now
    enable_progress_bar=False,
    enable_model_summary=False

)
nf = NeuralForecast(models=[model], freq='W-FRI')
nf.fit(df)

In [ ]:
preds = nf.predict()

print(preds.shape)
print(x_val.shape)

from src.utils import wmae

real = create_data_nbeats(x_val,y_val)
merged = pd.merge(real, preds, on=)
holidays = x_val['IsHoliday']
wmae(y_val,forecast.predict(x_val),holidays)

In [ ]:
# forecast.predict(x_val)

plt.figure(figsize=(12, 6))
plt.plot(x_val.Date, y_val, label='Weekly Sales val true', color='blue')
plt.plot(x_train.Date, y_train, label='Weekly Sales val true', color='red')
plt.plot(x_val.Date, forecast.NBEATS, label='Weekly Sales val predicted', color='green')
# plt.scatter(holidays['Date'], holidays['Weekly_Sales'], color='red', label='Holiday', zorder=5)

plt.title("Weekly Sales Over Time (Holidays Highlighted)")
plt.xlabel("Time (Weeks)")
plt.ylabel("Weekly Sales")
plt.legend()
plt.grid(True)
plt.show()